 * trailer: 
     * show a histogram with examples of distance / orientation with samples over ALL datasets
     * show perf!  show time to fetch frames using Spark + Parquet
     * show a video of one camera with debug overlays.  maybe one with delauny lidar too (!)
     * show a frame HTML with 3d interface
     * show new things: argo associated bikes, delauny lidar, occlusion tree
 * supported datasets, how to get a blurb and **stats** on each of them.  prolly render histo reports for each.
 * data structures:
    * StampedDatum
    * Frame

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/opt/psegs')

In [ ]:
from psegs.datasets import nuscenes

nuscenes.NuscStampedDatumTableLabelsAllFrames.build()




# from psegs.datasets import kitti

# kitti.DSUtil.test()

In [ ]:
from psegs.datasets import kitti_360

kitti_360.KITTI360SDTable.build()

In [ ]:
import sys
sys.path.append('/opt/psegs')
from oarphpy.spark import NBSpark
NBSpark.SRC_ROOT = '/opt/psegs'
NBSpark.SRC_ROOT_MODULES = ['psegs']
NBSpark.MAYBE_REBUILD_EGG_EVERY_CELL_RUN = False
NBSpark.CONF_KV = {
    'spark.driver.memory': '8g',
    'spark.pyspark.python': 'python3',
    'spark.python.worker.reuse': False,
    'spark.sql.files.maxPartitionBytes': int(8 * 1e6),
    'spark.port.maxRetries': '256',
  }
from psegs.datasets import kitti

spark = NBSpark.getOrCreate()
df = kitti.KITTISDTable.as_df(spark)
# df = spark.read.parquet('/tmp/avs_test/test_kitti_sd_table_tracking/sd_table/')
df.show(5)
# print(df.count())

In [ ]:
# seg = df.filter('uri.segment_id = "kitti-tracking-train-0009"')

In [ ]:
# seg_uri = seg.select('uri').persist()
df.createOrReplaceTempView('data')
# seg_uri.show()

In [ ]:
spark.sql("""
    SELECT
      uri.segment_id AS seg, 
      uri.topic AS topic,
      count(*) AS N,
      MAX(uri.timestamp),
      MIN(uri.timestamp),
      (MAX(uri.timestamp) - MIN(uri.timestamp)) * 1e-9 AS len
    FROM data
    GROUP BY topic, seg
    ORDER BY len ASC
""").show(1000, truncate=False)

In [ ]:
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=False)         # initiate notebook for offline plot

trace0 = Scatter(
  x=[1, 2, 3, 4],
  y=[10, 15, 13, 17]
)
trace1 = Scatter(
  x=[1, 2, 3, 4],
  y=[16, 5, 11, 9]
)

iplot([trace0, trace1])  


In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/opt/psegs')

In [ ]:
import psegs
from psegs.datasets import ios_lidar
from psegs import datum

from oarphpy.spark import NBSpark
# NBSpark.SRC_ROOT = '/opt/psegs'
NBSpark.SRC_ROOT_MODULES = ['psegs']
NBSpark.MAYBE_REBUILD_EGG_EVERY_CELL_RUN = True
NBSpark.CONF_KV = {
    'spark.driver.memory': '8g',
    'spark.pyspark.python': 'python3',
    'spark.python.worker.reuse': False,
    'spark.sql.files.maxPartitionBytes': int(8 * 1e6),
    'spark.port.maxRetries': '256',
  }


spark = NBSpark.getOrCreate()


suri = datum.URI.from_str(
      'psegs://dataset=psegs-ios-lidar-ext&split=threeDScannerApp_data&segment_id=charuco-test-fixture-lowres')
sd_df = ios_lidar.IOSLidarSDTable.as_df(spark, force_compute=True, only_segments=[suri])

sd_df.createTempView('sd_df')
sd_df.show()

In [ ]:
spark.sql("""
    SELECT 
      topic,
      n,
      (n / (1e-9 * duration_ns)) AS Hz,
      1e-9 * duration_ns AS duration_sec,
      width,
      height,
      channel_names,
      uncompressed_MBytes,
      uncompressed_MBytes / (1e-9 * duration_ns) AS uncompressed_MBps,
      FROM_UNIXTIME(start * 1e-9) AS start,
      FROM_UNIXTIME(end * 1e-9) AS end

    FROM 
        (
            SELECT
                FIRST(uri.topic) AS topic,
                MIN(uri.timestamp) AS start,
                MAX(uri.timestamp) AS end,
                MAX(uri.timestamp) - MIN(uri.timestamp) AS duration_ns,
                FIRST(camera_image.width) AS width,
                FIRST(camera_image.height) AS height,
                FIRST(camera_image.channel_names) AS channel_names,
                1e-6 * FIRST(camera_image.width * camera_image.height * 3) AS uncompressed_MBytes,
                COUNT(*) AS n
            FROM sd_df
            WHERE camera_image IS NOT NULL
            GROUP BY uri.topic
        )
    ORDER BY topic
""").toPandas()

In [ ]:
spark.sql("""
    SELECT 
      topic,
      n,
      (n / (1e-9 * duration_ns)) AS Hz,
      1e-9 * duration_ns AS duration_sec,
      cloud_colnames,
      FROM_UNIXTIME(start * 1e-9) AS start,
      FROM_UNIXTIME(end * 1e-9) AS end

    FROM 
        (
            SELECT
                FIRST(uri.topic) AS topic,
                MIN(uri.timestamp) AS start,
                MAX(uri.timestamp) AS end,
                MAX(uri.timestamp) - MIN(uri.timestamp) AS duration_ns,
                FIRST(point_cloud.cloud_colnames) AS cloud_colnames,
                COUNT(*) AS n
            FROM sd_df
            WHERE point_cloud IS NOT NULL
            GROUP BY uri.topic
        )
    ORDER BY topic
""").toPandas()

In [ ]:
spark.sql("""
    SELECT 
      topic,
      n,
      (n / (1e-9 * duration_ns)) AS Hz,
      1e-9 * duration_ns AS duration_sec,
      xform,
      FROM_UNIXTIME(start * 1e-9) AS start,
      FROM_UNIXTIME(end * 1e-9) AS end

    FROM 
        (
            SELECT
                FIRST(uri.topic) AS topic,
                MIN(uri.timestamp) AS start,
                MAX(uri.timestamp) AS end,
                MAX(uri.timestamp) - MIN(uri.timestamp) AS duration_ns,
                FIRST(CONCAT(transform.src_frame, '->', transform.dest_frame)) AS xform,
                COUNT(*) AS n
            FROM sd_df
            WHERE transform IS NOT NULL
            GROUP BY uri.topic
        )
    ORDER BY topic
""").toPandas()

In [ ]:
spark.sql("""
    SELECT 
      topic,
      n,
      (n / (1e-9 * duration_ns)) AS Hz,
      1e-9 * duration_ns AS duration_sec,
      FROM_UNIXTIME(start * 1e-9) AS start,
      FROM_UNIXTIME(end * 1e-9) AS end

    FROM 
        (
            SELECT
                FIRST(uri.topic) AS topic,
                MIN(uri.timestamp) AS start,
                MAX(uri.timestamp) AS end,
                MAX(uri.timestamp) - MIN(uri.timestamp) AS duration_ns,
                COUNT(*) AS n
            FROM sd_df
            WHERE SIZE(cuboids) > 0
            GROUP BY uri.topic
        )
    ORDER BY topic
""").toPandas()

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/opt/psegs')

import psegs
from psegs.datasets import ios_lidar
from psegs import datum

from oarphpy.spark import NBSpark
# NBSpark.SRC_ROOT = '/opt/psegs'
NBSpark.SRC_ROOT_MODULES = ['psegs']
NBSpark.MAYBE_REBUILD_EGG_EVERY_CELL_RUN = True
NBSpark.CONF_KV = {
    'spark.driver.memory': '8g',
    'spark.pyspark.python': 'python3',
    'spark.python.worker.reuse': False,
    'spark.sql.files.maxPartitionBytes': int(8 * 1e6),
    'spark.port.maxRetries': '256',
  }


spark = NBSpark.getOrCreate()


suri = datum.URI.from_str(
      'psegs://dataset=psegs-ios-lidar-ext&split=threeDScannerApp_data&segment_id=charuco-test-fixture-lowres')
sd_df = ios_lidar.IOSLidarSDTable.as_df(spark, force_compute=True, only_segments=[suri])

sd_df.createOrReplaceTempView('sd_df')
# sd_df.show()

In [ ]:


def get_depth_90th(row):
    ci = ios_lidar.IOSLidarSDTable.from_row(row.camera_image)
    depth = ci.get_depth()
    if depth is None:
        return 0
    else:
        import numpy as np
        return np.percentile(depth, 0.9)


    
    

sample_sd_df = spark.sql("""
                        SELECT 
                          *
                        FROM sd_df
                        WHERE uri.topic == 'camera|front'
                        ORDER BY RAND(1337)
                        LIMIT 50
                    """)

depth_top_90th = sample_sd_df.rdd.map(get_depth_90th).max()


if depth_top_90th <= 0.1:
    period_meters = 0.005
elif depth_top_90th <= 1.0:
    period_meters = 0.05
elif depth_top_90th <= 10.0:
    period_meters = 0.5
else:
    period_meters = 10.

print('period_meters', period_meters)
    
def to_t_debug_image(row):
    import cv2
    ci = ios_lidar.IOSLidarSDTable.from_row(row.camera_image)
    image = ci.get_debug_image(period_meters=period_meters)
#     if len(image.shape) != 3:
#         import numpy
#         image = np.tile([image, image, image], axis=2)
#     assert False, image.shape
    aspect = float(ci.width) / float(ci.height)
    target_height = 400
    target_width = int(aspect * target_height)
    
    # Pad the width a little to make ffmpeg most efficient
    if target_width % 16 != 0:
        target_width += 16 - (target_width % 16)
    
    image = cv2.resize(image, (target_width, target_height))

    return row.uri.timestamp, image



iter_t_image = sample_sd_df.rdd.map(to_t_debug_image).collect()
images = [image for t, image in sorted(iter_t_image, key=lambda ti: ti[0])]

im_min = min(i.min() for i in images)
print(im_min)
im_max = min(i.max() for i in images)
print(im_max)
images = [(255 * (i.astype('float') - im_min) / (im_max - im_min)).astype('uint8') for i in images]

from psegs.util.plotting import images_to_html_video

html = images_to_html_video(images, fps=4)
print('html size', 1e-6 * len(html))

def show_html(s):
    from IPython.core.display import display, HTML
    display(HTML(s), metadata=dict(isolated=True))

show_html(html)





In [ ]:
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=False)         # initiate notebook for offline plot


import plotly
import plotly.graph_objects as go


In [ ]:
sample_sd_df = spark.sql("""
                        SELECT 
                          *
                        FROM sd_df
                        WHERE uri.topic == 'camera|front|depth'
                        ORDER BY RAND(1337)
                        LIMIT 50
                    """)

def get_cloud_world(row):
    ci = ios_lidar.IOSLidarSDTable.from_row(row.camera_image)
    pc = ci.depth_image_to_point_cloud()
    cloud = pc.get_cloud()
    cloud = cloud[:, :3]
    T_world_from_ego = ci.ego_pose['ego', 'world']
    cloud_world = T_world_from_ego.apply(cloud).T
    return cloud_world

cloud_worlds = sample_sd_df.rdd.map(get_cloud_world).collect()

import numpy as np
import pandas as pd
cloud_df = None
for i, cloud in enumerate(cloud_worlds):
    color = int(255 * (float(i) / len(cloud_worlds)))
    
    to_sample = 1000
    if cloud.shape[0] > to_sample:
        idx = np.random.choice(np.arange(cloud.shape[0]), to_sample)
        cloud = cloud[idx, :]
    
    cur_df = pd.DataFrame(cloud, columns=['x', 'y', 'z'])
    cur_df['color'] = 'rgb(%s, %s, %s)' % (color, color, color)
    if cloud_df is None:
        cloud_df = cur_df
    else:
        cloud_df = pd.concat([cloud_df, cur_df])
    
plots = []


# cloud_df = pd.DataFrame(cloud_world, columns=['x', 'y', 'z'])
# from psegs.util.plotting import rgb_for_distance
# cloud_df['color'] = [
#   (128, 128, 128)#rgb_for_distance(np.linalg.norm(pt), period_meters=period_meters)
#   for pt in cloud_df[['x', 'y', 'z']].values
# ]
scatter = go.Scatter3d(
                x=cloud_df['x'], y=cloud_df['y'], z=cloud_df['z'],
                mode='markers',
                marker=dict(size=2, color=cloud_df['color'], opacity=0.5),)

plots.append(scatter)


fig = go.Figure(data=plots)

fig.update_layout(
  width=1000, height=700,
  scene_aspectmode='data')
  # scene_camera=dict(
  #   up=dict(x=0, y=0, z=1),
  #   eye=dict(x=0, y=0, z=0),
  #   center=dict(x=1, y=0, z=0),
  # ))
    
iplot(fig)

In [ ]:
sample_sd_df = spark.sql("""
                        SELECT 
                          *
                        FROM sd_df
                        WHERE uri.topic == 'camera|front'
                        ORDER BY RAND(1337)
                        LIMIT 50
                    """)

def get_t_ci(row):
    ci = ios_lidar.IOSLidarSDTable.from_row(row.camera_image)
    return row.uri.timestamp, ci

t_cis = sample_sd_df.rdd.map(get_t_ci).collect()
cis = [ci for t, ci in sorted(t_cis, key=lambda tc: tc[0])]

plots += [ci.to_plotly_world_frame_3d() for ci in cis]


fig = go.Figure(data=plots)

fig.update_layout(
  width=1000, height=700,
  scene_aspectmode='data')
  # scene_camera=dict(
  #   up=dict(x=0, y=0, z=1),
  #   eye=dict(x=0, y=0, z=0),
  #   center=dict(x=1, y=0, z=0),
  # ))
    
iplot(fig)



# def get_xzy_90ths(row):
#     ci = ios_lidar.IOSLidarSDTable.from_row(row.camera_image)
#     pc = ci.depth_image_to_point_cloud()
#     cloud = pc.get_cloud()
    
#     import numpy as np
#     return np.percentile(depth, 0.9)
    

# def to_t_pc_rv_debug_image(row):
#     ci = ios_lidar.IOSLidarSDTable.from_row(row.camera_image)
#     pc = ci.depth_image_to_point_cloud()
#     debug = pc.get_front_rv_debug_image()
    